In [2]:
from langchain_community.document_loaders import JSONLoader

In [6]:
from flask import Flask, request, jsonify
import json
import os
from langchain_core.documents import Document
# Text Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

app = Flask(__name__)

# File path for storing the data
DATA_FILE_PATH = 'webhook_data.json'

# Function to load data from the JSON file
def load_data():
    if os.path.exists(DATA_FILE_PATH):
        with open(DATA_FILE_PATH, 'r') as file:
            return json.load(file)
    else:
        return {}

# Function to save data to the JSON file
def save_data(data):
    with open(DATA_FILE_PATH, 'w') as file:
        json.dump(data, file, indent=4)  # Indent for better readability

@app.route('/webhook', methods=['POST'])
def handle_webhook():
    # Check if the request contains JSON data
    if request.is_json:
        data = request.get_json()
        
        # Process the webhook payload
        post_id = data.get('post_id')
        post_title = data.get('post_title')
        post_content = data.get('post_content')
        post_url = data.get('post_url')
        document = ([Document(page_content=post_content, metadata={"title": post_title, "post id": post_id})])
        print(document)
        # Create a RecursiveCharacterTextSplitter
        text_splitter = RecursiveCharacterTextSplitter(
            #separators = ["\n\n", "\n", " ", ""],    
            chunk_size = 500,
            chunk_overlap= 50,
            length_function=len
        )

            # Text splitting
        chunks = text_splitter.split_documents(documents=document)
        print(chunks)
        print(f"number of chunks: {len(chunks)}")
        # Load existing data
        existing_data = load_data()

        # Update existing data with new information
        existing_data[post_id] = {
            'post_title': post_title,
            'post_content': post_content,
            'post_url': post_url
        }

        # Save data to the JSON file
        save_data(existing_data)

        # Do something with the data (e.g., save it to a database)
        print(f"Received webhook for post {post_id}: {post_title}")

        # Send a response back to confirm receipt
        return jsonify({'message': 'Webhook received successfully'}), 200
    else:
        return jsonify({'error': 'Invalid JSON payload'}), 400

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

c:\Users\abhis\anaconda3\envs\genai\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
from langchain_community.vectorstores import Chroma

In [9]:
HTML='''!-- wp:heading {\"level\":1} -->\n<h1 class=\"wp-block-heading\" id=\"Beginner.27s_Guide_to_Python\">Beginner's Guide to Python</h1>\n<!-- /wp:heading -->\n\n<!-- wp:paragraph -->\n<p>New to programming? Python is free and easy to learn if you know where to start! This guide will help you to get started quickly.</p>\n<!-- /wp:paragraph -->\n\n<!-- wp:heading -->\n<h2 class=\"wp-block-heading\" id=\"New_to_Python.3F\">New to Python?</h2>\n<!-- /wp:heading -->\n\n<!-- wp:paragraph -->\n<p>Read&nbsp;<a href=\"https://wiki.python.org/moin/BeginnersGuide/Overview\">BeginnersGuide/Overview</a>&nbsp;for a short explanation of what Python is.</p>\n<!-- /wp:paragraph -->\n\n<!-- wp:heading -->\n<h2 class=\"wp-block-heading\" id=\"Getting_Python\">Getting Python</h2>\n<!-- /wp:heading -->\n\n<!-- wp:paragraph -->\n<p>Next, install the Python 3 interpreter on your computer. This is the program that reads Python programs and carries out their instructions; you need it before you can do any Python programming. Mac and Linux distributions may include an outdated version of Python (Python 2), but you should install an updated one (Python 3). See&nbsp;<a href=\"https://wiki.python.org/moin/BeginnersGuide/Download\">BeginnersGuide/Download</a>&nbsp;for instructions to download the correct version of Python.</p>\n<!-- /wp:paragraph -->\n\n<!-- wp:paragraph -->\n<p>There are also Python interpreter and IDE bundles available, such as&nbsp;<a href=\"http://thonny.org/\">Thonny</a>. Other options can be found at&nbsp;<a href=\"https://wiki.python.org/moin/IntegratedDevelopmentEnvironments\">IntegratedDevelopmentEnvironments</a>.</p>\n<!-- /wp:paragraph -->\n\n<!-- wp:paragraph -->\n<p>At some stage, you'll want to edit and save your program code. Take a look at&nbsp;<a href=\"https://wiki.python.org/moin/HowToEditPythonCode\">HowToEditPythonCode</a>&nbsp;for some advice and recommendations.</p>\n<!-- /wp:paragraph -->\n\n<!-- wp:heading -->\n<h2 class=\"wp-block-heading\" id=\"Learning_Python\">Learning Python</h2>\n<!-- /wp:heading -->\n\n<!-- wp:paragraph -->\n<p>Next, read a tutorial and try some simple experiments with your new Python interpreter.</p>\n<!-- /wp:paragraph -->\n\n<!-- wp:list -->\n<ul><!-- wp:list-item -->\n<li>If you have never programmed before, see\u00a0<a href=\"https://wiki.python.org/moin/BeginnersGuide/NonProgrammers\">BeginnersGuide/NonProgrammers</a>\u00a0for a list of suitable tutorials.</li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li>If you have previous programming experience, consult\u00a0<a href=\"https://wiki.python.org/moin/BeginnersGuide/Programmers\">BeginnersGuide/Programmers</a>, which lists more advanced tutorials.</li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li>If English isn't your first language, you might be more comfortable with a tutorial that's been translated into your language. Consult python.org's\u00a0<a href=\"https://wiki.python.org/moin/Languages\">list of Non-English resources</a>.</li>\n<!-- /wp:list-item --></ul>\n<!-- /wp:list -->\n\n<!-- wp:paragraph -->\n<p>Most tutorials assume you know how to run a program on your computer. If you are using Windows and need help with this, see&nbsp;<a href=\"http://www.python.org/doc/faq/windows/#how-do-i-run-a-python-program-under-windows\">How do I Run a Program Under Windows</a>.</p>\n<!-- /wp:paragraph -->\n\n<!-- wp:paragraph -->\n<p>Here are some sites that focus on beginners and offer in-browser coding:</p>\n<!-- /wp:paragraph -->\n\n<!-- wp:list -->\n<ul><!-- wp:list-item -->\n<li><a href=\"https://python.land/python-tutorial\">Beginners Python tutorial</a>\u00a0at Python Land (free)</li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li><a href=\"https://www.codedex.io/python\">Cod\u00e9dex</a>\u00a0(non-free)</li>\n<!-- /wp:list-item --></ul>\n<!-- /wp:list -->\n\n<!-- wp:list -->\n<ul><!-- wp:list-item -->\n<li><a href=\"https://www.coding-bootcamps.com/\">Coding Bootcamps</a>\u00a0(non-free)</li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li><a href=\"https://www.datacamp.com/\">DataCamp</a>\u00a0(non-free)</li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li><a href=\"https://www.dataquest.io/\">Dataquest</a>\u00a0for Python for data science. (free)</li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li><a href=\"https://www.hackinscience.org/\">HackInScience</a>\u00a0free and open source platform.</li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li><a href=\"https://www.myhsts.org/\">High School Technology Services</a>\u00a0for general Python (non-free)</li>\n<!-- /wp:list-item --></ul>\n<!-- /wp:list -->\n\n<!-- wp:paragraph -->\n<p>Print a&nbsp;<a href=\"https://blog.finxter.com/python-cheat-sheet/\">cheat sheet</a>&nbsp;of the most important Python features and post it to your office wall until you know the basics well.</p>\n<!-- /wp:paragraph -->\n\n<!-- wp:paragraph -->\n<p>Once you have read a tutorial, you can browse through&nbsp;<a href=\"http://docs.python.org/\">Python's online documentation</a>. It includes&nbsp;<a href=\"http://docs.python.org/tut/\">a tutorial</a>&nbsp;that might come in handy,&nbsp;<a href=\"http://docs.python.org//lib/\">a Library Reference</a>&nbsp;that lists all of the modules that come standard with Python, and&nbsp;<a href=\"http://docs.python.org/ref/\">the Language Reference</a>&nbsp;for a complete (if rather dry) explanation of Python's syntax.</p>\n<!-- /wp:paragraph -->\n\n<!-- wp:paragraph -->\n<p>When you are ready to write your first program, you will need a&nbsp;<a href=\"https://wiki.python.org/moin/PythonEditors\">text editor</a>&nbsp;or an&nbsp;<a href=\"https://wiki.python.org/moin/IntegratedDevelopmentEnvironments\">IDE</a>. If you don't want to use Thonny or something more advanced, then you can use&nbsp;<a href=\"https://docs.python.org/3/library/idle.html\">IDLE</a>, which is bundled with Python and supports&nbsp;<a href=\"http://idlex.sourceforge.net/\">extensions</a>.</p>\n<!-- /wp:paragraph -->\n\n<!-- wp:paragraph -->\n<p>This Python wiki also contains a page about&nbsp;<a href=\"https://wiki.python.org/moin/Powerful%20Python%20One-Liners\">Python One-Liners</a>&nbsp;-- an obscure but interesting subculture in Python.</p>\n<!-- /wp:paragraph -->\n\n<!-- wp:heading -->\n<h2 class=\"wp-block-heading\" id=\"Need_Help.3F\">Need Help?</h2>\n<!-- /wp:heading -->\n\n<!-- wp:paragraph -->\n<p>Need help with any of this? Read&nbsp;<a href=\"https://wiki.python.org/moin/BeginnersGuide/Help\">BeginnersGuide/Help</a>&nbsp;for mailing lists and newsgroups.</p>\n<!-- /wp:paragraph -->\n\n<!-- wp:paragraph -->\n<p>Most Python books will include an introduction to the language; see&nbsp;<a href=\"https://wiki.python.org/moin/IntroductoryBooks\">IntroductoryBooks</a>&nbsp;for suggested titles.</p>\n<!-- /wp:paragraph -->\n\n<!-- wp:paragraph -->\n<p>Consult&nbsp;<a href=\"https://wiki.python.org/moin/BeginnersGuide/Examples\">BeginnersGuide/Examples</a>&nbsp;for small programs and little snippets of code that can help you learn.</p>\n<!-- /wp:paragraph -->\n\n<!-- wp:paragraph -->\n<p>Or, if you prefer to learn Python through listening to a lecture, you can attend a training course or even hire a trainer to come to your company. Consult the&nbsp;<a href=\"https://wiki.python.org/moin/PythonEvents\">PythonEvents</a>&nbsp;page to see if any training courses are scheduled in your area and the&nbsp;<a href=\"https://wiki.python.org/moin/PythonTraining\">PythonTraining</a>&nbsp;page for a list of trainers.</p>\n<!-- /wp:paragraph -->\n\n<!-- wp:paragraph -->\n<p>Teachers can join the&nbsp;<a href=\"http://www.python.org/sigs/edu-sig/\">EDU-SIG</a>, a mailing list for discussion of Python's use in teaching at any level ranging from K-12 up to university.</p>\n<!-- /wp:paragraph -->\n\n<!-- wp:heading -->\n<h2 class=\"wp-block-heading\" id=\"Complete_list_of_Beginner.27s_Guide_pages\">Complete list of Beginner's Guide pages</h2>\n<!-- /wp:heading -->\n\n<!-- wp:list {\"ordered\":true,\"start\":1} -->\n<ol start=\"1\"><!-- wp:list-item -->\n<li><a href=\"https://wiki.python.org/moin/BeginnersGuide/Download\"><strong>BeginnersGuide/</strong>Download</a></li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li><a href=\"https://wiki.python.org/moin/BeginnersGuide/Examples\"><strong>BeginnersGuide/</strong>Examples</a></li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li><a href=\"https://wiki.python.org/moin/BeginnersGuide/Help\"><strong>BeginnersGuide/</strong>Help</a></li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li><a href=\"https://wiki.python.org/moin/BeginnersGuide/Mathematics\"><strong>BeginnersGuide/</strong>Mathematics</a></li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li><a href=\"https://wiki.python.org/moin/BeginnersGuide/NonProgrammers\"><strong>BeginnersGuide/</strong>NonProgrammers</a></li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li><a href=\"https://wiki.python.org/moin/BeginnersGuide/NonProgrammersChinese\"><strong>BeginnersGuide/</strong>NonProgrammersChinese</a></li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li><a href=\"https://wiki.python.org/moin/BeginnersGuide/Overview\"><strong>BeginnersGuide/</strong>Overview</a></li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li><a href=\"https://wiki.python.org/moin/BeginnersGuide/OverviewChinese\"><strong>BeginnersGuide/</strong>OverviewChinese</a></li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li><a href=\"https://wiki.python.org/moin/BeginnersGuide/Programmers\"><strong>BeginnersGuide/</strong>Programmers</a></li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li><a href=\"https://wiki.python.org/moin/BeginnersGuide/Programmers?action=AttachFile&amp;do=view&amp;target=Cpp2Python.pdf\">BeginnersGuide/Programmers\u00a0<strong>(Cpp2Python.pdf)</strong></a></li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li><a href=\"https://wiki.python.org/moin/BeginnersGuide/Programmers/SimpleExamples\"><strong>BeginnersGuide/</strong>Programmers/SimpleExamples</a></li>\n<!-- /wp:list-item --></ol>\n<!-- /wp:list -->\n\n<!-- wp:heading -->\n<h2 class=\"wp-block-heading\" id=\"Quiz_and_Exercises\">Quiz and Exercises</h2>\n<!-- /wp:heading -->\n\n<!-- wp:list -->\n<ul><!-- wp:list-item -->\n<li>Finxter -\u00a0<a href=\"https://finxter.com/\">How good are your Python skills? Test and Training with a Daily Python Puzzle</a></li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li>CheckIO -\u00a0<a href=\"http://www.checkio.org/\">Online learning, testing and improving your python skills</a></li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li>After Hours Programming -\u00a0<a href=\"http://www.afterhoursprogramming.com/tutorial/Python/Python-Quiz/\">Python Quiz</a></li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li>PyGUI -\u00a0<a href=\"http://www.pythongui.com/\">Collection of python quiz answers, Examples And GUI Tkinter Tutorials For Beginners</a></li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li>Pythonspot -\u00a0<a href=\"https://pythonspot.com/python-tests-quizes/\">Python Quiz</a></li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li>Python Challenge -\u00a0<a href=\"https://play.google.com/store/apps/details?id=sg.apps.garden.pythonchallenge\">A Python Quiz App on Android Platform</a></li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li><a href=\"http://cscircles.cemc.uwaterloo.ca/\">CS Circles - online lessons and graded exercises</a></li>\n<!-- /wp:list-item --></ul>\n<!-- /wp:list -->\n\n<!-- wp:heading -->\n<h2 class=\"wp-block-heading\" id=\"Python_Style_Checker\">Python Style Checker</h2>\n<!-- /wp:heading -->\n\n<!-- wp:list -->\n<ul><!-- wp:list-item -->\n<li><a href=\"http://pythonchecker.com/\">Pythonchecker.com</a>\u00a0- An educative online tool to rate your Python style (with dynamic score computation and hints)</li>\n<!-- /wp:list-item --></ul>\n<!-- /wp:list -->\n\n<!-- wp:heading -->\n<h2 class=\"wp-block-heading\" id=\"Looking_for_a_particular_Python_module_or_application.3F\">Looking for a particular Python module or application?</h2>\n<!-- /wp:heading -->\n\n<!-- wp:list -->\n<ul><!-- wp:list-item -->\n<li>The first place to look is the\u00a0<a href=\"http://pypi.python.org/pypi\">Python Package Index</a>.</li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li>If you can't find anything relevant in the Package Index,try\u00a0<a href=\"http://www.python.org/search/\">searching python.org</a>\u00a0- you can find anything mentioned on the Python site, in the\u00a0<a href=\"http://www.python.org/doc/faq/\">FAQs</a>, or in the newsgroup. More info:\u00a0<a href=\"http://www.python.org/search/\">where to search</a>.</li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li>You may also try our external guest project,\u00a0<a href=\"http://pydoc.net/\">pydoc.net</a>, for advanced package and module search.</li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li>Next, try\u00a0<a href=\"http://www.google.com/\">Google</a>\u00a0or another search engine of your choice. Searching for \"python\" and some relevant keywords will usually find something helpful.</li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li>Finally, you can try posting a query to the comp.lang.python Usenet group.</li>\n<!-- /wp:list-item --></ul>\n<!-- /wp:list -->\n\n<!-- wp:heading -->\n<h2 class=\"wp-block-heading\" id=\"Python-Related_Cheat_Sheets\">Python-Related Cheat Sheets</h2>\n<!-- /wp:heading -->\n\n<!-- wp:list -->\n<ul><!-- wp:list-item -->\n<li>Python:\u00a0<a href=\"https://blog.finxter.com/collection-5-cheat-sheets-every-python-coder-must-own/\">Collection of 11 Best Python Cheat Sheets</a></li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li><a href=\"https://wiki.python.org/moin/NumPy\">NumPy</a>:\u00a0<a href=\"https://blog.finxter.com/collection-10-best-numpy-cheat-sheets-every-python-coder-must-own/\">Collection of 10 Best NumPy Cheat Sheets</a></li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li>Pandas:\u00a0<a href=\"https://blog.finxter.com/pandas-cheat-sheets/\">Collection of 7 Beautiful Pandas Cheat Sheets</a></li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li>Machine Learning:\u00a0<a href=\"https://blog.finxter.com/machine-learning-cheat-sheets/\">Collection of 15 Machine Learning Cheat Sheets</a></li>\n<!-- /wp:list-item --></ul>\n<!-- /wp:list -->\n\n<!-- wp:heading -->\n<h2 class=\"wp-block-heading\" id=\"Want_to_contribute.3F\">Want to contribute?</h2>\n<!-- /wp:heading -->\n\n<!-- wp:list -->\n<ul><!-- wp:list-item -->\n<li>Python is a product of the\u00a0<a href=\"http://www.python.org/psf/\">Python Software Foundation</a>, a non-profit organization that holds the copyright.\u00a0<a href=\"http://www.python.org/psf/donations/\">Donations to the PSF</a>\u00a0are tax-deductible in the USA, and you can donate via credit card or\u00a0<a href=\"https://wiki.python.org/moin/PayPal\">PayPal</a>.</li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li>To report a bug in the Python core, use the\u00a0<a href=\"http://bugs.python.org/\">Python Bug Tracker</a>.</li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li>To contribute a bug fix or other patch to the Python core, read the\u00a0<a href=\"http://www.python.org/dev/\">Python Developer's Guide</a>\u00a0for more information about Python's development process.</li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li>To contribute to the official\u00a0<a href=\"http://www.python.org/doc/\">Python documentation</a>, join the\u00a0<a href=\"http://www.python.org/sigs/doc-sig/\">Documentation SIG</a>, write to\u00a0<a href=\"mailto:docs@python.org\">docs@python.org</a>\u00a0, or use the\u00a0<a href=\"http://bugs.python.org/\">Issue Tracker</a>\u00a0to contribute a documentation patch.</li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li>To announce your module or application to the Python community, use\u00a0<a href=\"news:comp.lang.python.announce\">comp.lang.python.announce</a>. See\u00a0<a href=\"http://www.python.org/community/lists/#comp-lang-python-announce\">the guide to Python mailing lists</a>\u00a0for more information.</li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li>To propose changes to the Python core, post your thoughts to\u00a0<a href=\"news:comp.lang.python\">comp.lang.python</a>. If you have an implementation, follow the\u00a0<a href=\"http://www.python.org/patches/\">Python Patch Guidelines</a>.</li>\n<!-- /wp:list-item -->\n\n<!-- wp:list-item -->\n<li>If you have a question are not sure where to report it, check out the\u00a0<a href=\"https://wiki.python.org/moin/WhereDoIReportThis%3F\">WhereDoIReportThis?</a>\u00a0page.</li>\n<!-- /wp:list-item --></ul>\n<!-- /wp:list -->",'''

In [10]:
type(HTML)

str

In [16]:
from langchain_community.document_transformers import Html2TextTransformer
import html2text

converter = html2text.HTML2Text()

# Convert HTML to plain text
plain_text = converter.handle(HTML)
plain_text 

'!-- wp:heading {"level":1} -->\n\n# Beginner\'s Guide to Python\n\nNew to programming? Python is free and easy to learn if you know where to\nstart! This guide will help you to get started quickly.\n\n## New to Python?\n\nRead\n[BeginnersGuide/Overview](https://wiki.python.org/moin/BeginnersGuide/Overview)\nfor a short explanation of what Python is.\n\n## Getting Python\n\nNext, install the Python 3 interpreter on your computer. This is the program\nthat reads Python programs and carries out their instructions; you need it\nbefore you can do any Python programming. Mac and Linux distributions may\ninclude an outdated version of Python (Python 2), but you should install an\nupdated one (Python 3). See\n[BeginnersGuide/Download](https://wiki.python.org/moin/BeginnersGuide/Download)\nfor instructions to download the correct version of Python.\n\nThere are also Python interpreter and IDE bundles available, such as\n[Thonny](http://thonny.org/). Other options can be found at\n[IntegratedDe

In [17]:
import dotenv

In [20]:
from dotenv import load_dotenv
import os
from pathlib import Path
load_dotenv()
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
def load_embedding(embedding_name):
    # Define environment variable names for different embeddings
    embedding_env_vars = {
        "openai": "OPENAI_API_KEY",
        "googleai": "GOOGLE_API_KEY"
    }

    # Check if the embedding name is valid
    if embedding_name not in embedding_env_vars:
        raise ValueError(f"Unsupported embedding: {embedding_name}")
    # Get the environment variable name for the given embedding
    env_var_name = embedding_env_vars[embedding_name]
    print(env_var_name)
    # Check if the environment variable exists
    if env_var_name not in os.environ:
        raise ValueError(f"Environment variable {env_var_name} not found")

    # Get the API key from the environment variable
    api_key = os.getenv[env_var_name]

    # Initialize vector embedding based on the embedding name
    if embedding_name == "openai":
        embeddings = OpenAIEmbeddings(api_key=api_key)
    elif embedding_name == "googleai":
        embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)
    else:
        raise ValueError(f"Unsupported embedding: {embedding_name}")
    return embeddings

# Example usage
try:
    openai_embeddings = load_embedding("openai")
    print("OpenAI embeddings loaded successfully")
except ValueError as e:
    print(e)

try:
    googleai_embeddings = load_embedding("googleai")
    print("GoogleAI embeddings loaded successfully")
except ValueError as e:
    print(e)

OPENAI_API_KEY


TypeError: 'function' object is not subscriptable